<a href="https://colab.research.google.com/github/prometricas/Abeba_Datos_Masivos/blob/main/mediaGolesPartido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Instalación de Java y Hadoop**

## **Instalar JDK de Java**

In [ ]:
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null

## **Instalar Hadoop**

In [ ]:
# Descargar Hadoop
!wget https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz

--2026-01-11 15:45:49--  https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1065831750 (1016M) [application/x-gzip]
Saving to: ‘hadoop-3.4.2.tar.gz’

hadoop-3.4.2.tar.gz 100%[===================>]   1016M  25.9MB/s    in 41s     

2026-01-11 15:46:31 (24.7 MB/s) - ‘hadoop-3.4.2.tar.gz’ saved [1065831750/1065831750]



In [ ]:
# Descomprimir y mover a carpeta de usuario
!tar -xzf hadoop-3.4.2.tar.gz
!mv hadoop-3.4.2/ /usr/local/

# Mostrar los archivos de carpeta local:
!ls /usr/local

bin    dist_metrics.pxd  games	       include	libexec     man  sbin	src
colab  etc		 hadoop-3.4.2  lib	LICENSE.md  opt  share


## **Definir variables de entorno**


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.4.2"
os.environ["PATH"] += os.pathsep + "/usr/local/hadoop-3.4.2/bin"
!hadoop version

Hadoop 3.4.2
Source code repository https://github.com/apache/hadoop.git -r 84e8b89ee2ebe6923691205b9e171badde7a495c
Compiled by ahmarsu on 2025-08-20T10:30Z
Compiled on platform linux-x86_64
Compiled with protoc 3.23.4
From source with checksum fa94c67d4b4be021b9e9515c9b0f7b6
This command was run using /usr/local/hadoop-3.4.2/share/hadoop/common/hadoop-common-3.4.2.jar


# **2. `Importar base de datos`**

In [ ]:
# Importar archivo
from google.colab import files
files.upload()

# Verificar
!ls -lah
!head -n 5 resultados_futbol.csv

Saving resultados_futbol.csv to resultados_futbol.csv
total 1018M
drwxr-xr-x 1 root root  4.0K Jan 11 15:47 .
drwxr-xr-x 1 root root  4.0K Jan 11 15:32 ..
-rw-r--r-- 1 root root   861 Jan 11 15:41 combinerMediaGolesPartido.py
drwxr-xr-x 4 root root  4.0K Dec 11 14:34 .config
-rw-r--r-- 1 root root 1017M Oct  4 09:30 hadoop-3.4.2.tar.gz
-rw-r--r-- 1 root root   978 Jan 11 15:41 mapperMediaGolesPartido.py
-rw-r--r-- 1 root root  1.3K Jan 11 15:43 reducerMediaGolesPartido.py
-rw-r--r-- 1 root root  557K Jan 11 15:47 resultados_futbol.csv
drwxr-xr-x 1 root root  4.0K Dec 11 14:34 sample_data
,Season,Game,Score,Teams,Home Team,Away Team
0,2000-2001,1,1-2,Mallorca - Real Madrid,Mallorca,Real Madrid
1,2000-2001,1,1-2,Valencia - Racing,Valencia,Racing
2,2000-2001,1,1-0,Athletic - Real Betis,Athletic,Real Betis
3,2000-2001,1,0-2,Atlético - Rayo Vallecano,Atlético,Rayo Vallecano


# **3. Crear scripts: *Mapper, Combiner, Reducer***


## **Mapper**
Emite por partido dos claves:
- **equipo|Local** con *(goles_local, 1)*
- **equipo|Visitante** con *(goles_visitante, 1)*

Usa una clave compuesta con '|' para que todo quede agrupado sin configurar separadores especiales.


In [ ]:
%%writefile mapperMediaGolesPartido.py
#!/usr/bin/python3
import sys
import csv
import re

pat_equipo = re.compile(r"\s*[-–]\s*")

for linea in sys.stdin:
    linea = linea.strip()
    if not linea:
        continue

    fila = next(csv.reader([linea]))

    if len(fila) >= 2 and fila[1].strip().lower() == "season":
        continue

    try:
        if len(fila) >= 7:
            score = fila[3].strip()
            local = fila[5].strip()
            visita = fila[6].strip()
        elif len(fila) >= 4:
            score = fila[2].strip()
            equipos = fila[3].strip()
            partes = pat_equipo.split(equipos, maxsplit=1)
            if len(partes) != 2:
                continue
            local, visita = partes[0].strip(), partes[1].strip()
        else:
            continue

        g_local, g_visita = [int(x.strip()) for x in score.split("-", 1)]

        print(f"{local}|Local\t{g_local}\t1")
        print(f"{visita}|Visitante\t{g_visita}\t1")

    except Exception:
        continue


Writing mapperMediaGolesPartido.py


## **Combiner**
* Acumula por clave **equipo|condición** las sumas de goles y la cantidad de partidos.
* Nunca calculo el promedio aquí, porque el promedio no es asociativo para combinar.



In [ ]:
%%writefile combinerMediaGolesPartido.py
#!/usr/bin/python3
import sys

clave_actual = None
goles_acum = 0
partidos_acum = 0

for linea in sys.stdin:
    linea = linea.strip()
    if not linea:
        continue

    partes = linea.split("\t")
    if len(partes) < 3:
        continue

    clave = partes[0]
    try:
        goles = int(partes[1])
        partidos = int(partes[2])
    except Exception:
        continue

    if clave_actual is None:
        clave_actual = clave
        goles_acum = goles
        partidos_acum = partidos
        continue

    if clave == clave_actual:
        goles_acum += goles
        partidos_acum += partidos
    else:
        print(f"{clave_actual}\t{goles_acum}\t{partidos_acum}")
        clave_actual = clave
        goles_acum = goles
        partidos_acum = partidos

if clave_actual is not None:
    print(f"{clave_actual}\t{goles_acum}\t{partidos_acum}")

Writing combinerMediaGolesPartido.py


## **Reducer**
* Suma goles y partidos por clave **equipo|condición** y calculo la media final.
* Yo emito: **equipo;condición;mediagoles**
* En este script, se ajusta el cálculo del promedio a números enteros aquí. Se hace aquí porque es donde más conviene por eficiencia, ya que *mapper* y *combiner* siguen trabajando con enteros (sumas) y en el *reducer* calculo el promedio final una sola vez por clave, evitando formateos/decimales: menos operaciones por registro.



In [ ]:
%%writefile reducerMediaGolesPartido.py
#!/usr/bin/python3
import sys

"""
Reducer: mediaGolesPartido (salida entera)
Yo sumo goles y partidos por clave equipo|condición y calculo la media final como entero.
Yo redondeo al entero más cercano usando solo aritmética entera para hacerlo más rápido.
Yo emito: equipo;condición;mediagoles
"""

clave_actual = None
goles_acum = 0
partidos_acum = 0

def emitir_resultado(clave, goles, partidos):
    if partidos <= 0:
        media_entera = 0
    else:
        # Yo hago redondeo al entero más cercano sin usar float: (goles + partidos/2) / partidos
        media_entera = (goles + (partidos // 2)) // partidos

    equipo, condicion = clave.split("|", 1)
    print(f"{equipo};{condicion};{media_entera}")

for linea in sys.stdin:
    linea = linea.strip()
    if not linea:
        continue

    partes = linea.split("\t")
    if len(partes) < 3:
        continue

    clave = partes[0]
    try:
        goles = int(partes[1])
        partidos = int(partes[2])
    except Exception:
        continue

    if clave_actual is None:
        clave_actual = clave
        goles_acum = goles
        partidos_acum = partidos
        continue

    if clave == clave_actual:
        goles_acum += goles
        partidos_acum += partidos
    else:
        emitir_resultado(clave_actual, goles_acum, partidos_acum)
        clave_actual = clave
        goles_acum = goles
        partidos_acum = partidos

if clave_actual is not None:
    emitir_resultado(clave_actual, goles_acum, partidos_acum)

Overwriting reducerMediaGolesPartido.py


## Permisos de ejecución
Dar permisos al usuario de consola para ejecutar cada archivo.

In [ ]:
!chmod u+x mapperMediaGolesPartido.py
!chmod u+x combinerMediaGolesPartido.py
!chmod u+x reducerMediaGolesPartido.py

# **4. Hadoop Streaming**
Se fuerza 1 reducer para que la salida mantenga el orden global por clave (alfabético).

In [ ]:
!rm -rf salidaMediaGolesPartido

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.4.2.jar \
  -D mapreduce.job.reduces=1 \
  -file ./mapperMediaGolesPartido.py -mapper ./mapperMediaGolesPartido.py \
  -file ./combinerMediaGolesPartido.py -combiner ./combinerMediaGolesPartido.py \
  -file ./reducerMediaGolesPartido.py -reducer ./reducerMediaGolesPartido.py \
  -input resultados_futbol.csv -output ./salidaMediaGolesPartido


2026-01-11 15:54:04,322 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./mapperMediaGolesPartido.py, ./combinerMediaGolesPartido.py, ./reducerMediaGolesPartido.py] [] /tmp/streamjob10531852711797292657.jar tmpDir=null
2026-01-11 15:54:05,428 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2026-01-11 15:54:05,707 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-11 15:54:05,748 INFO mapreduce.JobSubmitter: number of splits:1
2026-01-11 15:54:06,129 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local450223745_0001
2026-01-11 15:54:06,131 INFO mapreduce.JobSubmitter: Executing with tokens: []
2026-01-11 15:54:06,578 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapperMediaGolesPartido.py as file:/tmp/hadoop-root/mapred/local/job_local450223745_0001_1bed036b-b388-4c54-ac4c-5ca9c9346b80/mapperMediaGolesPartido.py
2026-01-11 15:54:06,629 INFO mapr

In [ ]:
!cat ./salidaMediaGolesPartido/part-*

Alavés;Local;1	
Alavés;Visitante;1	
Albacete;Local;1	
Albacete;Visitante;1	
Almería;Local;1	
Almería;Visitante;1	
Athletic;Local;1	
Athletic;Visitante;1	
Atlético;Local;2	
Atlético;Visitante;1	
Barcelona;Local;3	
Barcelona;Visitante;2	
Celta;Local;1	
Celta;Visitante;1	
Cádiz;Local;1	
Cádiz;Visitante;1	
Córdoba;Local;1	
Córdoba;Visitante;1	
Deportivo Alavés;Local;1	
Deportivo Alavés;Visitante;1	
Deportivo;Local;2	
Deportivo;Visitante;1	
Eibar;Local;1	
Eibar;Visitante;1	
Elche;Local;1	
Elche;Visitante;1	
Espanyol;Local;1	
Espanyol;Visitante;1	
Getafe;Local;1	
Getafe;Visitante;1	
Gimnàstic Tarragona;Local;1	
Gimnàstic Tarragona;Visitante;1	
Girona;Local;1	
Girona;Visitante;1	
Granada;Local;1	
Granada;Visitante;1	
Huesca;Local;1	
Huesca;Visitante;1	
Hércules;Local;1	
Hércules;Visitante;0	
Las Palmas;Local;1	
Las Palmas;Visitante;1	
Leganés;Local;1	
Leganés;Visitante;1	
Levante;Local;1	
Levante;Visitante;1	
Mallorca;Local;1	
Mallorca;Visitante;1	
Málaga;Local;1	
Málaga;Visitante;1	
Numancia